# 车牌定位系统实验

本notebook用于进行车牌定位系统的实验和调试。

In [ ]:
import sys
sys.path.append('..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from src.config.config import Config
from src.preprocessing.noise_reduction import gaussian_filter, median_filter
from src.preprocessing.illumination import histogram_equalization, adaptive_histogram_equalization
from src.methods.color_based.hsv_detector import HSVDetector
from src.methods.edge_based.sobel_detector import SobelDetector
from src.methods.deep_learning.yolo_detector import YOLODetector
from src.postprocessing.filter import CandidateFilter
from src.postprocessing.fusion import ResultFusion
from src.utils.visualization import draw_boxes, show_results

## 1. 数据加载与预处理

In [ ]:
def load_test_image(image_path):
    """加载测试图像"""
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# 加载测试图像
test_image_path = '../data/car2024/test1.jpg'
image = load_test_image(test_image_path)

plt.figure(figsize=(12, 8))
plt.imshow(image)
plt.axis('off')
plt.title('原始图像')
plt.show()

## 2. 预处理效果测试

In [ ]:
# 测试不同预处理方法
def show_preprocessing_results(image):
    # 高斯滤波
    gaussian_result = gaussian_filter(image)
    
    # 中值滤波
    median_result = median_filter(image)
    
    # 直方图均衡化
    hist_result = histogram_equalization(image)
    
    # 自适应直方图均衡化
    adaptive_hist_result = adaptive_histogram_equalization(image)
    
    # 显示结果
    plt.figure(figsize=(15, 10))
    
    plt.subplot(231)
    plt.imshow(image)
    plt.title('原始图像')
    plt.axis('off')
    
    plt.subplot(232)
    plt.imshow(gaussian_result)
    plt.title('高斯滤波')
    plt.axis('off')
    
    plt.subplot(233)
    plt.imshow(median_result)
    plt.title('中值滤波')
    plt.axis('off')
    
    plt.subplot(234)
    plt.imshow(hist_result)
    plt.title('直方图均衡化')
    plt.axis('off')
    
    plt.subplot(235)
    plt.imshow(adaptive_hist_result)
    plt.title('自适应直方图均衡化')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

show_preprocessing_results(image)

## 3. 单一方法测试

In [ ]:
def test_single_method(image, method='hsv'):
    # 预处理
    processed_image = gaussian_filter(image)
    processed_image = adaptive_histogram_equalization(processed_image)
    
    # 选择检测方法
    if method == 'hsv':
        detector = HSVDetector()
    elif method == 'sobel':
        detector = SobelDetector()
    elif method == 'yolo':
        detector = YOLODetector()
    
    # 检测
    boxes = detector.detect(processed_image)
    
    # 后处理
    candidate_filter = CandidateFilter()
    filtered_boxes = candidate_filter.apply_filters(boxes)
    
    # 可视化
    result_image = draw_boxes(processed_image, filtered_boxes)
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(121)
    plt.imshow(processed_image)
    plt.title('预处理图像')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(result_image)
    plt.title(f'{method.upper()} 检测结果')
    plt.axis('off')
    
    plt.show()

# 测试HSV方法
test_single_method(image, 'hsv')

# 测试Sobel方法
test_single_method(image, 'sobel')

# 测试YOLO方法
test_single_method(image, 'yolo')

## 4. 结果融合测试

In [ ]:
def test_fusion(image):
    # 预处理
    processed_image = gaussian_filter(image)
    processed_image = adaptive_histogram_equalization(processed_image)
    
    # 多方法检测
    detections = {}
    
    # HSV检测
    hsv_detector = HSVDetector()
    hsv_boxes = hsv_detector.detect(processed_image)
    detections['hsv'] = (hsv_boxes, [0.8] * len(hsv_boxes))
    
    # Sobel检测
    sobel_detector = SobelDetector()
    sobel_boxes = sobel_detector.detect(processed_image)
    detections['sobel'] = (sobel_boxes, [0.7] * len(sobel_boxes))
    
    # YOLO检测
    yolo_detector = YOLODetector()
    yolo_boxes = yolo_detector.detect(processed_image)
    detections['yolo'] = (yolo_boxes, [0.9] * len(yolo_boxes))
    
    # 结果融合
    fusion = ResultFusion()
    final_boxes = fusion.weighted_fusion(detections)
    
    # 后处理
    candidate_filter = CandidateFilter()
    final_boxes = candidate_filter.apply_filters(final_boxes)
    
    # 可视化最终结果
    result_image = draw_boxes(processed_image, final_boxes)
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(121)
    plt.imshow(image)
    plt.title('原始图像')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(result_image)
    plt.title('融合后的最终结果')
    plt.axis('off')
    
    plt.show()

test_fusion(image)

## 5. 性能评估

In [ ]:
from src.utils.metrics import evaluate_detection

def evaluate_methods(image, gt_boxes):
    methods = ['hsv', 'sobel', 'yolo', 'fusion']
    results = {}
    
    for method in methods:
        if method == 'fusion':
            # 使用融合方法
            detections = {}
            hsv_detector = HSVDetector()
            sobel_detector = SobelDetector()
            yolo_detector = YOLODetector()
            
            hsv_boxes = hsv_detector.detect(image)
            sobel_boxes = sobel_detector.detect(image)
            yolo_boxes = yolo_detector.detect(image)
            
            detections['hsv'] = (hsv_boxes, [0.8] * len(hsv_boxes))
            detections['sobel'] = (sobel_boxes, [0.7] * len(sobel_boxes))
            detections['yolo'] = (yolo_boxes, [0.9] * len(yolo_boxes))
            
            fusion = ResultFusion()
            pred_boxes = fusion.weighted_fusion(detections)
        else:
            # 使用单一方法
            pred_boxes = test_single_method(image, method, return_boxes=True)
        
        # 评估结果
        metrics = evaluate_detection(pred_boxes, gt_boxes)
        results[method] = metrics
    
    # 显示评估结果
    for method, metrics in results.items():
        print(f"\n{method.upper()} 方法评估结果:")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1-Score: {metrics['f1']:.4f}")

# 示例标注框（需要根据实际数据调整）
gt_boxes = [
    np.array([[100, 100], [200, 100], [200, 150], [100, 150]])
]

evaluate_methods(image, gt_boxes)